# Wrangling OpenStreetMap Data with MongoDB

# Overview

Location: Las Vegas, NV <br>
Link: https://mapzen.com/data/metro-extracts/metro/las-vegas_nevada/

In this project, I will be downloading data from OpenStreetMap and analyzing the data with MongoDB.  The location I chose to perform this analysis on is the city in which I am planning to visit in a few weeks, Las Vegas, NV.  Analyzing this  data will invlove me analyzing the data from an .OMS file and cleaning for use in as a .JSON file.  Once the file is converted, I will analyze the data in the MongoDB created.

# Importing information for analysis

In [1]:
import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
import sys
import os
from collections import defaultdict
sys.path.append('./scripts')

data_name = "las-vegas_nevada"
OSMFILE = "{}.osm".format(data_name)

# 1. Problems Encountered in the Map
# Tag Problems

With open crowd source data, for obvious reasons, there are some corrections that need to be made.  In this overview, I will describe the process I took to look through and clean the data once it was downloaded from OpenStreetMap.org.  The first step will be looking throught tag problems to point out just how errors fall within the data

**The problems are defined as follows:**
*   "lower" - for tags that contain only lowercase letters and are valid,
*   "upper" - for tags that contain upercase letters,
*   "lower_colon" - for otherwise valid tags with a colon in their names,
*   "uper_colon" - for tags contain both uppercase strings delimited by a colon,
*   "multiple_colons" - for tags contain more than two colon seperated strings,
*   "numbers" - for tags that contain a digit, and
*   "problemchars" - for tags with problematic characters
*   "other" - for other tags that do not fall into the other three categories.

In [2]:
import tags as tags_processor
tag_problems = tags_processor.process_map(OSMFILE)

print "The number of keys in each of the 'problem' categories:"
print tag_problems['counts']
unique_key_names = tags_processor.unique_tag_keys(OSMFILE)
print "There are {} unique tag key names in the data set.".format(len(unique_key_names))

The number of keys in each of the 'problem' categories:
{'problemchars': 0, 'upper': 572, 'lower': 303285, 'upper_colon': 457, 'numbers': 6908, 'multiple_colons': 218, 'lower_colon': 263107, 'other': 148}
There are 792 unique tag key names in the data set.


Zero "problemchars" and 148 in the "other" category is great!  This means that the data will flow into the other categories nicely without a hitch.  Lets move to the next step.

# Audit Street Names

In [3]:
import audit as street_name_auditor
street_name_auditor.audit(OSMFILE)

defaultdict(set,
            {'1': {'Spanish Ridge Ave., Suite 1'},
             '10': {'S. Valley View Blvd. Ste 10'},
             '100': {'S Eastern Ave #100'},
             '103': {'South Pecos Road Suite 103'},
             '106': {'S Grand Canyon Dr #106'},
             '107': {'S Decatur Blvd #107'},
             '11': {'Chandler Ave #11'},
             '110': {'South Jones Boulevard Suite 110'},
             '115': {'North Hualapai Way #115'},
             '1170': {'Fremont Street Ste. 1170'},
             '120': {'E SILVERADO RANCH BLVD Suite 120',
              'Silverado Ranch Blvd #120'},
             '15': {'E Sahara Blvd #15'},
             '150': {'West Horizon Ridge Parkway, STE 150'},
             '16': {'Polaris Ave #16'},
             '170': {'Sky Pointe Dr #170'},
             '207': {'E Sahara Ave #207'},
             '275': {'Camino Al Norte #275'},
             '2D': {'Spring Mountain Rd #2D'},
             '3230': {'W Sahara Ave #3230'},
             '500': {'Ho

As you can see, there are some issues with the data above, there are a few ways that the audit function can pick up certain address data.  In the example above, the street name's are mostly correct.  Moving forward, correcting entries have been added to the JSON creation code below to correct any errors.  Next, I will move into checking the cuisine area, and if few errors are found, it is time to create a JSON file to import into MongoDB!

# Cuisine Check

In [4]:
import cuisine as cuisine_auditor
food_nodes = cuisine_auditor.audit(OSMFILE)

food_nodes_with_cuisine_and_amenity = [n for n in food_nodes if 'cuisine' in n and 'amenity' in n]
food_nodes_without_cuisine = [n for n in food_nodes if 'cuisine' not in n]
food_nodes_without_amenity = [n for n in food_nodes if 'amenity' not in n]
print "Number of food nodes: {}".format(len(food_nodes))
print "Number of food nodes with a cuisine and amenity: {}".format(len(food_nodes_with_cuisine_and_amenity))
print "Number of food nodes without a cuisine: {}".format(len(food_nodes_without_cuisine))
print "Number of food nodes without an amenity: {}".format(len(food_nodes_without_amenity))

Number of food nodes: 992
Number of food nodes with a cuisine and amenity: 435
Number of food nodes without a cuisine: 557
Number of food nodes without an amenity: 0


To check the accuracy of the cuisine data, I performaed the above check on the cuisine.  This is a cound of all cuisine not with or without a cuisine or amenity.  The findings show that there are 992 food nodes, 435 have both cuisine and amenity, 557 food nodes do not have a cuisine, and all food nodes have an amenity!

In [5]:
from collections import Counter

amenities = []
for node in food_nodes_without_cuisine:
    amenities.append(node['amenity'])

print "Amenity counts for food nodes without a cuisine:"
print Counter(amenities)

Amenity counts for food nodes without a cuisine:
Counter({'restaurant': 251, 'fast_food': 139, 'bar': 81, 'cafe': 62, 'pub': 24})


This code performs a search for amenity and counts the type of food amenities without a cuisine.  As you can see, restaurants and fast food are the categories with the most missing cuisine and make up 70% of the food nodes without a cuisine. The 557 nodes that have food related amenities but lack a cuisine are broken down by amenity as follows:
- restaurant: 251
- fast_food: 139
- bar: 81 
- cafe: 62 
- pub: 24

# Zip and Phone Audit and Correct

In [6]:
postal_code_range = [89000,89600]
postal_code_default = 89000

def audit_postal_code(invalid_postal_codes, postal_code):
    try:
        if not (postal_code_range[0] <= int(postal_code) <= postal_code_range[1]):
            raise ValueError
    except ValueError:
        invalid_postal_codes[postal_code] += 1

def audit_phone_number(invalid_phone_numbers, phone_number):
    try:
        if len(phone_number) != 12 or phone_number[:3] != '+1':
            raise ValueError
    except ValueError:
        invalid_phone_numbers[phone_number] += 1

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def is_phone_number(elem):
    return (elem.attrib['k'] == "phone")

def audit(OSMFILE):
    osm_file = open(OSMFILE, "r")
    invalid_postal_codes = defaultdict(int)
    invalid_phone_numbers = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(invalid_postal_codes, tag.attrib['v'])
                elif is_phone_number(tag):
                    audit_phone_number(invalid_phone_numbers, tag.attrib['v'])

    return [invalid_postal_codes, invalid_phone_numbers]

audit(OSMFILE)

[defaultdict(int,
             {'6451112': 1,
              '89014-2132': 1,
              '89102-4370': 1,
              '89104-1307': 1,
              '89108-7044': 1,
              '89108-7049': 1,
              '89109-1907': 1,
              '891171': 1,
              '89119-1001': 1,
              '89119-6304': 1,
              u'89123\u200e': 1,
              '89128-6634': 1,
              '89135-1020': 1,
              '89135-1038': 1,
              '89146-2977': 1,
              '89147-4111': 1,
              '89147-8491': 1,
              '8929': 1,
              'NV 89014': 1,
              'NV 89030': 1,
              'NV 89031': 1,
              'NV 89052': 2,
              'NV 89101': 1,
              'NV 89107': 1,
              'NV 89109': 2,
              'NV 89117': 1,
              'NV 89119': 2,
              'NV 89123': 1,
              'NV 89124': 1,
              'NV 89129': 1,
              'NV 89134': 2,
              'NV 89135': 2,
              'NV 89142': 1,


All errors noted in the cuisine check have been noted, and corrections have been made in the code below.  This exercise was simply done to explore data before creating the .JSON file. This analysis shows that there are definite problems with the cuisine classifications of food nodes within the data.  Let's create a JSON now!

# Create the New .JSON file from the .OSM file

In [7]:
#checks if postal code within valid range, if not replaces with 11000 default
def update_postal_code(postal_code):
    try:
        if not (postal_code_range[0] <= int(postal_code) <= postal_code_range[1]):
            raise ValueError
        else:
            return int(postal_code)
    except ValueError:
        return postal_code_default

#standardizes phone number formatting
def update_phone_number(phone_number):
    phone_number = phone_number.translate(None, ' ()-+')
    phone_number = '+1' + phone_number[-10:]
    return phone_number

#standardizes street types with a replacement map
def update_name(name, mapping):
    name = name.split(' ')
    type = name[-1]
    if type in mapping:
        name[-1] = mapping[type]
    
    name = ' '.join(name)
    name = name.title()

    return name

In [8]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = ["version", "changeset", "timestamp", "user", "uid"]

street_mapping = {
    ',': '',
    '?': '',
    "'": '',
    ".": "",
    'Avenue 1': '',
    'Avenue 2': '',
    'Avenue 3': '',
    'Avenue 4': '',
    'road': 'Road',
    'way': 'Way',
    'Ave': 'Avenue',
    'Rd': 'Road',
    'St': 'Street',
    'St.': 'Street',
    'street':'Street',
    'Rd': 'Road',
    'Rd.': 'Road',
    'Ln':'Lane',}

def shape_element(e):
    node = {}
    node['created'] = {}
    node['pos'] = [0,0]
    if e.tag == "way":
        node['node_refs'] = []
    if e.tag == "node" or e.tag == "way" :
        node['type'] = e.tag
        #attributes
        for k, v in e.attrib.iteritems():
            #latitude
            if k == 'lat':
                try:
                    lat = float(v)
                    node['pos'][0] = lat
                except ValueError:
                    pass
            #longitude
            elif k == 'lon':
                try:
                    lon = float(v)
                    node['pos'][1] = lon
                except ValueError:
                    pass
            #creation metadata
            elif k in CREATED:
                node['created'][k] = v
            else:
                node[k] = v
        #children
        for tag in e.iter('tag'):
            k = tag.attrib['k']
            v = tag.attrib['v']
            if problemchars.match(k):
                continue
            elif lower_colon.match(k):
                k_split = k.split(':')
                #address fields
                if k_split[0] == 'addr':
                    k_item = k_split[1]
                    if 'address' not in node:
                        node['address'] = {}
                    #streets
                    if k_item == 'street':
                        v = update_name(v, street_mapping)                    
                    #postal codes
                    if k_item == 'postcode':
                        v = update_postal_code(v)
                    node['address'][k_item] = v
                    continue
            else:                
                #phone numbers
                if(is_phone_number(tag)):
                    v = update_phone_number(v)

            node[k] = v
        #way children
        if e.tag == "way":
            for n in e.iter('nd'):
                ref = n.attrib['ref']
                node['node_refs'].append(ref);
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def test():
    data = process_map(OSMFILE, True)
    # pprint.pprint(data)


if __name__ == "__main__":
    test()

# Import to MongoDB

Once this JSON file is created, I now opened up terminal and imported the file we just created into the MongoDB.  See below for command line promt entry

# 2. Data Overview

## File Sizes

In [9]:
# Size of the OSM and JSON files.
import os
def get_size_in_mb_of_relative_file(file_name):
    wd = %pwd
    return os.stat(wd + '/' + file_name).st_size / 1000.0 / 1000.0

for file_name in [OSMFILE, OSMFILE + '.json']:
    file_size = get_size_in_mb_of_relative_file(file_name)
    print "{} is {} MB in size.".format(file_name, file_size)

las-vegas_nevada.osm is 222.586422 MB in size.
las-vegas_nevada.osm.json is 322.967605 MB in size.


Now, after all that work, lets find the name and the street of a thai restaurant with outdoor seating.  Shouldn't be too hard to accomplish with mongodb!

In [10]:
def get_db(db_name):
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client[db_name]
    return db

def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$match" : {"amenity" : "restaurant",
                             "cuisine" : "thai",
                             "outdoor_seating"  : "yes"}},
                {"$project" : {"Street" : "$address.street",
                              "Name" : "$name"}},
                {"$sort" : {"street" : -1}},
                {"$limit" : 1}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.vegas.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('project')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)

[{u'Name': u'Le Thai',
  u'Street': u'Fremont Street',
  u'_id': ObjectId('58e82083f3a9edf3ca12bb5f')}]


Return the number of documents

In [11]:
db.vegas.find().count()

1120030

Return the number of Node's

In [12]:
db.vegas.find({"type":"node"}).count()

1015938

Number of Ways

In [13]:
db.vegas.find({"type":"way"}).count()

103930

Top 5 contributing users

In [14]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{ "$group" : {"_id" : "$created.user",
                             "count" : { "$sum" : 1}}},
                    { "$sort" : {"count" : -1}}, 
                    {"$limit":5}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.vegas.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('project')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)

[{u'_id': u'alimamo', u'count': 253281},
 {u'_id': u'tomthepom', u'count': 121172},
 {u'_id': u'woodpeck_fixbot', u'count': 73365},
 {u'_id': u'alecdhuse', u'count': 66574},
 {u'_id': u'abellao', u'count': 56819}]


Number of users who only have one post

In [15]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$group":{"_id":"$created.user", "count":{"$sum":1}}}, 
                {"$group":{"_id":"$count", "num_users":{"$sum":1}}}, 
                {"$sort":{"_id":1}}, 
                {"$limit":1}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.vegas.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('project')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)

[{u'_id': 1, u'num_users': 236}]


Looking at all of this data you will notice, that there are a hige amount of submissions from the top 5 contributors.  I decided to look at some statistics for these members:
 * **The Top Contributor** - The top contributor was alimamo who contributed 253,281 submissions.  This is 23% of the data!
 * **The Top 5 Contributors** - The top 5 contributors who contributed 571,211 submissions.  This is 51% of the data!

With this amount of contributions, this makes you weary and hope that these contributors entered in the correct data in this open source map of Vegas.

# 3. Additional Ideas

Top 10 amenities

In [16]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$match":{"amenity":{"$exists":1}}}, 
                {"$group":{"_id":"$amenity","count":{"$sum":1}}}, 
                {"$sort":{"count":-1}}, 
                {"$limit":10}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.vegas.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('project')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)

[{u'_id': u'parking', u'count': 997},
 {u'_id': u'school', u'count': 534},
 {u'_id': u'restaurant', u'count': 507},
 {u'_id': u'place_of_worship', u'count': 377},
 {u'_id': u'fast_food', u'count': 293},
 {u'_id': u'fountain', u'count': 282},
 {u'_id': u'fuel', u'count': 196},
 {u'_id': u'toilets', u'count': 99},
 {u'_id': u'cafe', u'count': 88},
 {u'_id': u'bar', u'count': 84}]


#1 Religion

In [17]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"place_of_worship"}}, 
                {"$group":{"_id":"$religion", "count":{"$sum":1}}}, 
                {"$sort":{"count":-1}}, 
                {"$limit":1}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.vegas.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('project')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)


[{u'_id': u'christian', u'count': 352}]


And just for fun, lets look at the top cuisines and see what we are working with

In [18]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{"$match":{"amenity":{"$exists":1}, "amenity":"restaurant"}}, 
                {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
                {"$sort":{"count":-1}}, 
                {"$limit":5}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.vegas.aggregate(pipeline)]

if __name__ == '__main__':
    db = get_db('project')
    pipeline = make_pipeline()
    result = aggregate(db, pipeline)
    import pprint
    pprint.pprint(result)

[{u'_id': None, u'count': 253},
 {u'_id': u'mexican', u'count': 43},
 {u'_id': u'pizza', u'count': 35},
 {u'_id': u'american', u'count': 21},
 {u'_id': u'italian', u'count': 19}]


# Conclusion

After this review of the data it’s obvious that the Vegas area is incomplete, though I believe it has been well cleaned for the purposes of this exercise. However, the improvement may bring about potential problems if it's not implemented correctly:

1.Gamifimation may impact the quality (veracity) of the data submitted from the contributors. We need to keep in mind that quality should always be considered more important than quantity when we try implementing the improvement. 
2.If the difference between the highest score and the rest is too large, users may easily loose their interest. Therefore, we should implement it in such a way that the higher the score is, the harder it becomes to increase. 

It interests me to notice a fair amount of GPS data makes it into OpenStreetMap.org on account of users’ efforts. With a rough GPS data processor in place and working together with a more robust data processor similar to data.py, I think it would be possible to input great amount of cleaned data to OpenStreetMap.org